# The basics

### Step 1. Powers of two and binary length

A power of two looks like:

$$
2^0, \; 2^1, \; 2^2, \; 2^3, \; \dots
$$

If $n$ is between $2^k$ and $2^{k+1}$, then the smallest power of two ≥ $n$ is $2^{k+1}$.

So our task reduces to:
**Given $n$, find the smallest integer exponent $k$ such that $2^k \geq n$.**

---

### Step 2. Take base-2 logs

Taking $\log_2$ on both sides:

$$
2^k \geq n \quad \iff \quad k \geq \log_2(n).
$$

That means the **smallest valid $k$** is:

$$
k = \lceil \log_2(n) \rceil
$$

(ceiling = round up to the next integer).

---

### Step 3. Reconstruct the power

Once you have $k$, the next power of two is just:

$$
2^k = 1 \ll k \quad \text{(bit-shift form in Python)}
$$

or equivalently:

```python
1 << math.ceil(math.log2(n))
```

---

### Step 4. Example

Take $n = 9$:

* $\log_2(9) \approx 3.17$.
* Ceiling: $\lceil 3.17 \rceil = 4$.
* $2^4 = 16$.
  Indeed, 16 is the smallest power of two ≥ 9.

Take $n = 16$:

* $\log_2(16) = 4.0$.
* Ceiling: 4.
* $2^4 = 16$.
  It stays at 16 (not 32), which is correct.

---

### Step 5. Why ceiling is crucial

If you used `floor` instead, for $n = 9$:

* $\lfloor \log_2(9) \rfloor = 3$.
* $2^3 = 8 < 9$.
  Too small. That’s why you must round **up**.

In [1]:
    
import math

def next_power_of_two(n: int) -> int:
    if n <= 1:
        return 1
    # this is 2**math.ceil(math.log2(n))
    return 1 << math.ceil(math.log2(n))

print(next_power_of_two(10))


16


### Array–Binary Tree Indexing Rules

We store a complete binary tree in a flat array (0-based indexing). The mapping between **parent** and **child** indices comes from how nodes are laid out level by level.

---

#### Parent rule

For a node at **index** $j > 0$:

$$
\text{parent}(j) \;=\; \left\lfloor \frac{j-1}{2} \right\rfloor
$$

* Subtracting 1 “undoes” the +1 or +2 that was added when computing children.
* Dividing by 2 (integer floor) moves one level up in the binary tree.

---

#### Children rules

For a node at **index** $i$:

$$
\text{left}(i)  \;=\; 2i + 1, \qquad
\text{right}(i) \;=\; 2i + 2
$$

* Multiplying by 2 doubles the index because each level doubles the number of nodes.
* Adding +1 or +2 selects the first (left) or second (right) child slot.

---

#### Visual example

Array indices:

$$
[\,0,\; 1,\; 2,\; 3,\; 4,\; 5,\; 6\,]
$$

Tree structure:

```
          0
        /   \
       1     2
      / \   / \
     3   4 5   6
```

Check the formulas:

* For $i=0$:
  $\text{left}(0)=1,\; \text{right}(0)=2$.
* For $i=1$:
  $\text{left}(1)=3,\; \text{right}(1)=4$.
* For $i=2$:
  $\text{left}(2)=5,\; \text{right}(2)=6$.
* For $j=4$:
  $\text{parent}(4)=\lfloor(4-1)/2\rfloor=1$.

---

#### Why multiply by 2?

Each level of a complete binary tree has **twice as many nodes** as the previous one:

* Level 0: $1$ node → index 0
* Level 1: $2$ nodes → indices 1–2
* Level 2: $4$ nodes → indices 3–6
* Level 3: $8$ nodes → indices 7–14
  and so on.

So when you descend from node $i$ to its children, you “jump” into the next block of nodes — exactly $2i$ away, plus an offset (1 for left, 2 for right).

# Main code

In [2]:
def tournament_sort(arr):
    """
    Main tournament sort function.
    Returns a new sorted list, leaving original unchanged.
    """
    if len(arr) <= 1:
        return arr[:]

    # Build the tournament tree
    tree = build_tournament(arr)

    # Extract elements one by one
    result = []
    for _ in range(len(arr)):
        min_val = extract_min(tree)
        if min_val != float('inf'):  # Don't include padding values
            result.append(min_val)

    return result


def build_tournament(arr):
    """
    Build the tournament tree from input array.
    Returns a list representing the complete binary tree.
    Each node is a tuple: (value, original_leaf_index)
    """
    n = len(arr)
    print(f"Building tournament tree for array of size {n}")
    leaf_count = next_power_of_two(n)
    print(f"Next power of two (leaf count): {leaf_count}")

    # Calculate total nodes in complete binary tree
    total_nodes = 2 * leaf_count - 1

    # Initialize tree with placeholder values
    tree = [(float('inf'), -1)] * total_nodes

    # Calculate starting index for leaf level
    leaf_start = total_nodes - leaf_count

    # Fill in the leaves (pad with infinity for unused slots)
    for i in range(leaf_count):
        if i < n:
            tree[leaf_start + i] = (arr[i], i)
        else:
            tree[leaf_start + i] = (float('inf'), i)

    # Build internal nodes bottom-up
    # Start from the level just above leaves
    for i in range(leaf_start - 1, -1, -1):
        left_child = 2 * i + 1
        right_child = 2 * i + 2

        # Choose minimum with left-bias for stability
        tree[i] = tree[left_child] if tree[left_child][0] <= tree[right_child][0] else tree[right_child]

    return tree

def extract_min(tree):
    """
    Extract one minimum from the tournament tree and update it in-place.

    High-level steps:
      1) Read the root: it holds the global minimum and the winning leaf index.
      2) Invalidate the winning leaf by setting its value to +∞.
      3) Recompute ONLY the nodes on the path from that leaf back to the root
         (O(log n) updates).
      4) Return the extracted minimum value.

    Returns:
      - The extracted minimum value, or +∞ if the tree is already exhausted.
    """
    # Empty tree or already exhausted (root is +∞) → nothing to extract
    if not tree or tree[0][0] == float('inf'):
        return float('inf')

    # --- Step 1: Root has the current global minimum and its winning leaf index ---
    #   Node format: (value, original_leaf_index)
    min_val, winning_leaf = tree[0]

    # --- Step 2: Compute the position of that leaf in the flat array layout ---
    # The last level contains all leaves. For a complete binary tree with L leaves,
    # total_nodes = 2*L - 1, so leaves occupy indices [total_nodes - L ... total_nodes - 1].
    leaf_count = (len(tree) + 1) // 2             # L
    leaf_start = len(tree) - leaf_count           # first leaf slot index
    leaf_pos   = leaf_start + winning_leaf        # specific leaf that won at the root

    # --- Step 3: Invalidate the winning leaf (remove it from future competition) ---
    # We keep the same leaf index tuple but set value to +∞.
    tree[leaf_pos] = (float('inf'), winning_leaf)

    # --- Step 4: Update only the path from this leaf up to the root (O(log n)) ---
    # Parent arithmetic for array-based binary tree:
    #   parent = (child_index - 1) // 2
    # Children indices of a parent p are (2*p + 1) and (2*p + 2).
    current = leaf_pos
    while current > 0:
        parent      = (current - 1) // 2
        left_child  = 2 * parent + 1
        right_child = 2 * parent + 2

        # Recompute the parent as the min (stable left-bias) of its two children
        tree[parent] = tree[left_child] if tree[left_child][0] <= tree[right_child][0] else tree[right_child]

        # Climb one level up and continue until we reach the root
        current = parent

    # The tree is now consistent again; the new root is the next global minimum.
    return min_val


In [3]:
def build_walkthrough(arr):
    """
    Print a pedagogical, step-by-step BUILD of the tournament tree for `arr`.
    """
    inf = float('inf')
    n = len(arr)
    
    L = next_power_of_two(n)
    total = 2*L - 1
    print(f"total nodes = 2*{L} - 1 = {total}")
    leaf_start = total - L
    print("leaf_start =", leaf_start)

    print(f"n={n}, leaf_count={L}, total_nodes={total}, leaf_start={leaf_start}")
    tree = [(inf, -1)] * total
    

    # Fill leaves
    print("\n-- Fill leaves --")
    for k in range(L):
        val = arr[k] if k < n else inf
        tree[leaf_start + k] = (val, k)
        val_str = "∞" if val == inf else str(val)
        note = "padding" if k >= n else f"arr[{k}]"
        print(f"tree[{leaf_start + k}] = ({val_str}, {k})   # {note}")

    # Build internal nodes bottom-up
    print("\n-- Build internal nodes (bottom-up) --")
    for i in range(leaf_start - 1, -1, -1):
        lc, rc = 2*i + 1, 2*i + 2
        Lnode, Rnode = tree[lc], tree[rc]
        # Lnode = (val, idx), Rnode = (val, idx)
        val_lnode, idx_lnode = Lnode
        val_rnode, idx_rnode = Rnode
        chosen = Lnode if (val_lnode < val_rnode or
                          (val_lnode == val_rnode)) else Rnode
        tree[i] = chosen
        def fmt(t): return f'({"∞" if t[0]==inf else t[0]}, {t[1]})'
        print(f"i={i:>2d}: children [{lc},{rc}] = {fmt(Lnode)}, {fmt(Rnode)}  ->  tree[{i}] = {fmt(chosen)}")

    root_val, root_leaf = tree[0]
    rv = "∞" if root_val == inf else root_val
    print(f"\nROOT = tree[0] = ({rv}, {root_leaf})  # global minimum at the root")
    return tree



def print_tree(tree):
    """
    Helper function to visualize the tree structure.
    Useful for debugging.
    """
    if not tree:
        return

    leaf_count = (len(tree) + 1) // 2
    height = leaf_count.bit_length()

    level_start = 0
    for level in range(height):
        level_size = 1 << level  # 2^level
        level_end = min(level_start + level_size, len(tree))

        print(f"Level {level}:", end=" ")
        for i in range(level_start, level_end):
            val, idx = tree[i]
            if val == float('inf'):
                print("(∞,-)", end=" ")
            else:
                print(f"({val},{idx})", end=" ")
        print()

        level_start = level_end
        if level_start >= len(tree):
            break

In [4]:
test_arrays = [
    [64, 34, 25, 12, 22, 11, 90],
    [5, 2, 8, 1, 9],
    [1],
    [],
    [3, 3, 3, 3],
    [9, 8, 7, 6, 5, 4, 3, 2, 1]
]

for i, arr in enumerate(test_arrays):
    print(f"\nTest {i+1}: {arr}")

    # Build tree and show structure
    if arr:
        tree = build_walkthrough(arr)
        print("Initial tree:")
        print_tree(tree)

    # Sort and compare with Python's built-in sort
    sorted_arr = tournament_sort(arr)
    expected = sorted(arr)

    print(f"Tournament sort: {sorted_arr}")
    print(f"Expected:        {expected}")
    print(f"Correct: {sorted_arr == expected}")

    if arr and len(arr) <= 8:  # Show step-by-step for small arrays
        print("\nStep-by-step extraction:")
        tree = build_walkthrough(arr)
        for step in range(len(arr)):
            min_val = extract_min(tree)
            print(f"  Step {step+1}: extracted {min_val}")


Test 1: [64, 34, 25, 12, 22, 11, 90]
total nodes = 2*8 - 1 = 15
leaf_start = 7
n=7, leaf_count=8, total_nodes=15, leaf_start=7

-- Fill leaves --
tree[7] = (64, 0)   # arr[0]
tree[8] = (34, 1)   # arr[1]
tree[9] = (25, 2)   # arr[2]
tree[10] = (12, 3)   # arr[3]
tree[11] = (22, 4)   # arr[4]
tree[12] = (11, 5)   # arr[5]
tree[13] = (90, 6)   # arr[6]
tree[14] = (∞, 7)   # padding

-- Build internal nodes (bottom-up) --
i= 6: children [13,14] = (90, 6), (∞, 7)  ->  tree[6] = (90, 6)
i= 5: children [11,12] = (22, 4), (11, 5)  ->  tree[5] = (11, 5)
i= 4: children [9,10] = (25, 2), (12, 3)  ->  tree[4] = (12, 3)
i= 3: children [7,8] = (64, 0), (34, 1)  ->  tree[3] = (34, 1)
i= 2: children [5,6] = (11, 5), (90, 6)  ->  tree[2] = (11, 5)
i= 1: children [3,4] = (34, 1), (12, 3)  ->  tree[1] = (12, 3)
i= 0: children [1,2] = (12, 3), (11, 5)  ->  tree[0] = (11, 5)

ROOT = tree[0] = (11, 5)  # global minimum at the root
Initial tree:
Level 0: (11,5) 
Level 1: (12,3) (11,5) 
Level 2: (34,1) (12,

# Extract step-by-step

In [5]:
def extract_walkthrough(tree):
    """
    Pedagogical walkthrough for ONE extraction:
      - Shows the root winner,
      - Shows the leaf invalidation,
      - Shows each parent update along the path to the root.

    Prints a short log and returns the extracted value.
    """
    inf = float('inf')

    # Guard for exhausted tree
    if not tree or tree[0][0] == inf:
        print("root is ∞ → nothing to extract")
        return inf

    # Root winner
    min_val, winning_leaf = tree[0]
    print(f"[emit] root = (value={min_val}, leaf={winning_leaf})")

    # Locate leaf slots
    L          = (len(tree) + 1) // 2
    leaf_start = len(tree) - L
    leaf_pos   = leaf_start + winning_leaf

    # Invalidate leaf
    tree[leaf_pos] = (inf, winning_leaf)
    print(f"[invalidate] leaf at tree[{leaf_pos}] ← (∞, {winning_leaf})")

    # Update path to root
    current = leaf_pos
    print("[update path]")
    while current > 0:
        parent      = (current - 1) // 2
        left_child  = 2 * parent + 1
        right_child = 2 * parent + 2

        before = tree[parent]
        newval = tree[left_child] if tree[left_child][0] <= tree[right_child][0] else tree[right_child]
        tree[parent] = newval

        def fmt(node):
            v, i = node
            return f"(∞,{i})" if v == inf else f"({v},{i})"

        print(f"  parent tree[{parent}] : min( "
              f"tree[{left_child}]={fmt(tree[left_child])}, "
              f"tree[{right_child}]={fmt(tree[right_child])} ) "
              f"  {fmt(before)} → {fmt(newval)}")

        current = parent

    # After updates, tree[0] is the next global min for the following extract.
    print(f"[done] extracted {min_val}\n")
    return min_val


from copy import deepcopy

arr = [64, 34, 25, 12, 22, 11, 90]
tree = build_tournament(arr)  # your function returns just `tree`
print_tree(tree)
# Make a copy if you want to keep the original for multiple trials
T = deepcopy(tree)

# One pedagogical extract:
extract_walkthrough(T)
# Next extract (optional, to see another path update):
extract_walkthrough(T)



Building tournament tree for array of size 7
Next power of two (leaf count): 8
Level 0: (11,5) 
Level 1: (12,3) (11,5) 
Level 2: (34,1) (12,3) (11,5) (90,6) 
Level 3: (64,0) (34,1) (25,2) (12,3) (22,4) (11,5) (90,6) (∞,-) 
[emit] root = (value=11, leaf=5)
[invalidate] leaf at tree[12] ← (∞, 5)
[update path]
  parent tree[5] : min( tree[11]=(22,4), tree[12]=(∞,5) )   (11,5) → (22,4)
  parent tree[2] : min( tree[5]=(22,4), tree[6]=(90,6) )   (11,5) → (22,4)
  parent tree[0] : min( tree[1]=(12,3), tree[2]=(22,4) )   (11,5) → (12,3)
[done] extracted 11

[emit] root = (value=12, leaf=3)
[invalidate] leaf at tree[10] ← (∞, 3)
[update path]
  parent tree[4] : min( tree[9]=(25,2), tree[10]=(∞,3) )   (12,3) → (25,2)
  parent tree[1] : min( tree[3]=(34,1), tree[4]=(25,2) )   (12,3) → (25,2)
  parent tree[0] : min( tree[1]=(25,2), tree[2]=(22,4) )   (12,3) → (22,4)
[done] extracted 12



12